In [23]:
from perceval.backends.core.github import GitHub
from datetime import date
import datetime
import dateutil.tz
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy as s
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker
import augur
from time import gmtime, strftime

In [24]:
strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())

'Wed, 01 Aug 2018 21:04:05 +0000'

In [16]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect,list1,path = augurApp.github_issues()
token = list1[5]
DB_STR = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(
            list1[0], list1[1], list1[2],\
            list1[3], list1[4]
        )
db = s.create_engine(DB_STR)

In [17]:
table_names = s.inspect(db).get_table_names()
print(table_names)
val = False
#mail_lists = True
git_repos = True
numb = 0
if("github_issues_2" not in table_names):
    #mail_lists = False
    git_repos = False
if("github_repo_jobs" in table_names):
    lists_createdSQL = s.sql.text("""SELECT project FROM github_repo_jobs""")
    df1 = pd.read_sql(lists_createdSQL, db)
    print(df1)
    val = True
    val = db.engine.execute("""SELECT augurlistID FROM 
                                   mailing_list_jobs
                                   ORDER BY augurlistID DESC LIMIT 1""")
    for i in val:
        numb = i['augurlistID']
    
else:
    columns2 = "augurlistID","backend_name","github_repo_url","project","last_date"
    df_mail_list = pd.DataFrame(columns=columns2)
    df_mail_list.to_sql(name="github_repo_jobs",con=db,if_exists='replace',index=False,
                        dtype={'augurlistID': s.types.Integer(),
                               'backend_name': s.types.VARCHAR(length=300),
                               'github_repo_url': s.types.VARCHAR(length=300),
                               'project': s.types.VARCHAR(length=300),
                               'last_message_date': s.types.DateTime()
                        })
    lists_createdSQL = s.sql.text("""SELECT project FROM github_repo_jobs""")
    df1 = pd.read_sql(lists_createdSQL, db)
    db.execute("ALTER TABLE github_repo_jobs ADD PRIMARY KEY (augurlistID)")
    print("Created Table")
Base = declarative_base(db)
class table(Base):
    __tablename__ = 'github_repo_jobs'
    __table_args__={'autoload':True}
    
Session = sessionmaker(bind=db)
session = Session()
res = session.query(table).all()

['github_issues', 'github_issues_sentiment_scores', 'github_pull_requests', 'github_pull_requests_sentiment_scores', 'github_repo_jobs', 'issue_response_time', 'mail_lists', 'mail_lists_sentiment_scores', 'mailing_list_jobs']
Empty DataFrame
Columns: [project]
Index: []


In [18]:
if "notebooks" in os.getcwd():
    os.chdir("..")
connect,list1,path = augurApp.github_issues()
print(path)
if(not os.path.exists(path)):
    file = open(path, "w+")
else:
    file = open(path, "r")
    print("yeah")
if (os.stat(path).st_size == 0):
    file.write("owner,repo_url\n")
    file.write("nodejs,\"CTC\"\n")
    #file.write("https://lists.opendaylight.org/pipermail/,\"archetypes-dev\"\n")
    print("Please enter the mailing lists and the links for them please")
    print("Going to the default mailing lists")

count = 0
for line in file:
    print(line)
    count+=1
    if(count == 2):
        break

runtime/git_repos.csv
yeah
owner,repo_url

torvalds,"linux"



In [19]:
df5 = pd.read_csv(path)

In [20]:
df5

,owner,repo_url
0,torvalds,linux
1,nodejs,CTC
2,nodejs,TSC


In [22]:
github_repos = []
for i in range(len(df5)):
    github_repos.append(df5.loc[i,'owner'] + '/' + df5.loc[i,'repo_url'])
print(github_repos)
github_check = {key:False for key in github_repos}
print(github_check)
    

['torvalds/linux', 'nodejs/CTC', 'nodejs/TSC']
{'torvalds/linux': False, 'nodejs/CTC': False, 'nodejs/TSC': False}


In [4]:
# Url for the git repo to analyze
#repo_url = 'https://github.com/chaoss/grimoirelab-perceval'
#repo_url = 'CTC'
# Directory for letting Perceval clone the git repo
#repo_dir = 'CTC'
#own = 'nodejs'
# ElasticSearch instance (url)
#repo = 'grimoirelab-perceval'

# Create the 'commits' index in ElasticSearch
# Create a Git object, pointing to repo_url, using repo_dir for cloning

#32
#date = datetime.datetime(2017, 8, 7, 0, 0, 0,
#                                     tzinfo=dateutil.tz.tzutc())
k=0
#for issues in repo.fetch(from_date=date):
columns1 = 'augurmsgID',"backend_name",'repo_link',"owner","repo","subject",\
          "status","category","issue_number","timestamp",\
          "issue_id","user","body"
df = pd.DataFrame(columns=columns1)
item = 1
df.to_sql(name="github_issues_2", con=db,\
          if_exists='replace',index=False,
           dtype={'augurmsgID': s.types.Integer,#0
                  'backend_name': s.types.VARCHAR(length=300),#1
                  'repo_link': s.types.VARCHAR(length=300),#2
                  'owner': s.types.VARCHAR(length=300),#3
                  'repo': s.types.VARCHAR(length=300),#4
                  'subject': s.types.VARCHAR(length=300),#5
                  'status': s.types.VARCHAR(length=10),#6
                  'category': s.types.VARCHAR(length=10),#7
                  'issue_number': s.types.Integer,#8
                  'timestamp': s.types.Integer,#9
                  'issue_id': s.types.Integer,#10
                  'user': s.types.VARCHAR(length=100),#11
                  'body':s.types.TEXT#12              
           })

groups = df5.groupby('owner').groups
x = 0
for group in groups:
    own = group
    repo_url = (df5.loc[df5['owner'] == group]['repo_url']).tolist()
    for y in repo_url:        
        repo = GitHub(owner=own,repository=y,api_token=token)
        for issues in repo.fetch():
            # Create the object (dictionary) to upload to ElasticSearch
            # Create the object (dictionary) to upload to ElasticSearch
            #if(i==90):
            if 'pull_request' in issues['data']:
                #print(issues['data']['number'])
                continue
            else:
                #if(issues['data']['state'] == 'closed' and issues['data']\
                #   ['number'] == 165): 
                text = issues['data']['body']
                user = issues['data']['user']['login']
                num = issues['data']['number']
                id_num = issues['data']['id']
                #print(id_num)
                store = [item,issues['backend_name'],issues['tag'],own,y,\
                         issues['data']['title'],issues['data']['state'],\
                         issues['category'],num,issues['timestamp'],\
                         id_num,user,text]
                #print(store[:])
                #print(store[6])
                #print("Issue number: ",issues['data']['number'])
                #if(k > 1):
                    #k+=1
                    #continue
                '''print("Backend_name: ", issues['backend_name'])
                print("Category: ", issues['category'])
                print("Subject: ", issues['data']['title'])
                print("State: ", issues['data']['state'])
                print("First body:",issues['data']['body'])
                print("User: ", issues['data']['user']['login'])
                print("\n\n\n")'''
                df_user = pd.DataFrame([store],columns=columns1)
                df = df.append(df_user)
                item+=1
                for i in range(len(issues['data']['comments_data'])):
                    temp=0
                    store[0] = item
                    store[8] = issues['data']['number']
                    store[10] = issues['data']['id']
                    store[11] = issues['data']['comments_data'][i]\
                    ['user_data']['login']
                    store[12] = issues['data']['comments_data'][i]['body']
                    df_user = pd.DataFrame([store],columns=columns1)
                    df = df.append(df_user)
                    #print(issues['data']['comments_data'][i]\
                    #      ['user_data']['login'])
                    #print(issues['data']['comments_data'][i]['body'])
                    item+=1

            if(df.shape[0] > 1000):
                df.to_sql(name='github_issues_2', con=db,\
                          if_exists='append',index=False)
                df = pd.DataFrame(columns=columns1)
                print("Broken")
                #break
            #print(issues['data'][''])
            if(k%50 == 0):
                print(k)
            k+=1
            #if(k == 2):
            #    break
        #print(df)
        #print(df)
        if(df.shape[0] < 1000):
            df.to_sql(name='github_issues_2', con=db,\
                          if_exists='append',index=False)
            df = pd.DataFrame(columns=columns1)
            print("Broken")
        print("Total Number of issues: ", k)
        print(df.shape[0])

2018-07-25 11:47:39 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/GeorgLink
2018-07-25 11:47:40 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/germonprez


0


2018-07-25 11:47:40 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/downeym
2018-07-25 11:47:41 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/aserebrenik
2018-07-25 11:47:42 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/jgbarah
2018-07-25 11:47:42 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/aswanipranjal
2018-07-25 11:47:43 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/jsdabrowski
2018-07-25 11:47:44 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/f2014169
2018-07-25 11:47:44 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/jaishreedhage
2018-07-25 11:47:44 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/apoorvkhare07
2018-07-25 11:47:46 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api

Broken
Total Number of issues:  10
0


2018-07-25 11:48:03 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/rvagg
2018-07-25 11:48:03 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/vkurchatkin
2018-07-25 11:48:04 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/benjamingr
2018-07-25 11:48:04 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/Trott
2018-07-25 11:48:04 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/joshgav
2018-07-25 11:48:05 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/Fishrock123
2018-07-25 11:48:06 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/jasnell
2018-07-25 11:48:06 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/addaleax
2018-07-25 11:48:07 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/C

...

2018-07-25 12:06:56 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/golya87
2018-07-25 12:06:56 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/Mcsiv
2018-07-25 12:06:56 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/shark300
2018-07-25 12:06:57 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/hron84
2018-07-25 12:06:57 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/sushantbajra
2018-07-25 12:06:58 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/ixaxaar
2018-07-25 12:06:58 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/Jonatino
2018-07-25 12:06:58 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/vikaskyadav
2018-07-25 12:06:59 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/r

2018-07-25 12:07:28 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/bholagabbar
2018-07-25 12:07:28 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/jobs-git
2018-07-25 12:07:29 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/CodeFetch
2018-07-25 12:07:30 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/PommyGit
2018-07-25 12:07:30 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/gregv21v
2018-07-25 12:07:31 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/shayneoAtNorwood
2018-07-25 12:07:31 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/hwpplayer1
2018-07-25 12:07:31 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/christ2go
2018-07-25 12:07:32 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.git

2018-07-25 12:08:01 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/tuxayo
2018-07-25 12:08:02 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/ple103
2018-07-25 12:08:02 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/Pierre-Sassoulas
2018-07-25 12:08:03 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/tuckerpo
2018-07-25 12:08:04 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/ammarfaizi2
2018-07-25 12:08:04 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/chrisfinazzo
2018-07-25 12:08:05 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/daxtens
2018-07-25 12:08:05 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/yankunsam
2018-07-25 12:08:06 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.

2018-07-25 12:08:40 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/alonWang
2018-07-25 12:08:40 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/findstr
2018-07-25 12:08:41 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/rod-lin
2018-07-25 12:08:41 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/kevin-y-ma
2018-07-25 12:08:42 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/YaoyaoChang
2018-07-25 12:08:42 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/Art2Cat
2018-07-25 12:08:43 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/windlessStorm
2018-07-25 12:08:43 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/users/sunuslee
2018-07-25 12:08:43 keanu-Inspiron-5567 root[11710] INFO Getting info for https://api.github.com/

RateLimitError: Rate limit exhausted.; 2473.0 seconds to rate reset